This is for the search dropdown menu in the manual tab, adds to the RUFF database

In [16]:
import sqlite3
import pandas as pd
import json
import math

# File paths
csv_file = 'MPdatabase.csv'  # Replace with your CSV file path
database_name = 'RRUFFRaman_databaseSEARCH.db'  # Existing database

# Table name in the database
table_name = 'microplastics_data'

# Load the CSV file into a pandas DataFrame
data = pd.read_csv(csv_file)

# Replace "NA" with None (interpreted as NULL in SQLite)
data.replace("NA", None, inplace=True)

try:
    # Connect to the existing SQLite database with a timeout
    conn = sqlite3.connect(database_name, timeout=10)
    cursor = conn.cursor()

    # Ensure table schema is correct, x_data and y_data as REAL
    cursor.execute(f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        filename TEXT UNIQUE,
        x_data REAL,
        y_data REAL
    );
    """)

    # Prepare data for insertion
    x_values = data['freq'].tolist()  # x_data (shared across all columns)
    for column_name in data.columns[1:]:  # Loop through each y-data column
        y_values = data[column_name].tolist()
    
        # Remove None (NULL) or NaN values from both x_data and y_data together
        cleaned_data = [(x, y) for x, y in zip(x_values, y_values) 
                        if x is not None and not (isinstance(x, float) and math.isnan(x)) 
                        and y is not None and not (isinstance(y, float) and math.isnan(y))]
    
        # Unzip the cleaned data back into x_values_clean and y_values_clean
        x_values_clean, y_values_clean = zip(*cleaned_data) if cleaned_data else ([], [])
    
        # Convert x_data and y_data into JSON strings
        x_data_json = json.dumps(x_values_clean)
        y_data_json = json.dumps(y_values_clean)
    
        # Insert or update row for the current filename
        cursor.execute(f"""
        INSERT INTO {table_name} (filename, x_data, y_data)
        VALUES (?, ?, ?)
        ON CONFLICT(filename) DO UPDATE SET
        x_data = excluded.x_data,
        y_data = excluded.y_data;
        """, (column_name, x_data_json, y_data_json))
    
    # Commit changes
    conn.commit()
    print(f"Data from '{csv_file}' added to '{database_name}' in table '{table_name}' successfully.")


except sqlite3.OperationalError as e:
    print(f"An error occurred: {e}")

finally:
    # Close the connection
    if 'conn' in locals():
        conn.close()


Data from 'MPdatabase.csv' added to 'RRUFFRaman_databaseSEARCH.db' in table 'microplastics_data' successfully.


now for the matching database

In [2]:
import sqlite3
import pandas as pd
import math

# File paths
csv_file = 'MPdatabase.csv'  # Replace with your CSV file path
database_name = 'RRUFFRaman_database.db'  # Existing database

# Table name in the database
table_name = 'microplastics_data_match'

# Load the CSV file into a pandas DataFrame
data = pd.read_csv(csv_file)

# Replace "NA" with None (interpreted as NULL in SQLite)
data.replace("NA", None, inplace=True)

try:
    # Connect to the existing SQLite database with a timeout
    conn = sqlite3.connect(database_name, timeout=10)
    cursor = conn.cursor()

    # Drop the existing table if it exists
    cursor.execute(f"DROP TABLE IF EXISTS {table_name};")

    # Create the table schema to match the database_table schema
    cursor.execute(f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        id INTEGER PRIMARY KEY,
        filename TEXT,
        mineral_name TEXT,
        rruff_id TEXT,
        wavelength TEXT,
        orientation TEXT,
        file_number TEXT,
        elements TEXT,
        x_data REAL,
        y_data REAL
    );
    """)

    # Loop through each column (filename is in the column name, and freq is the x_data)
    x_values = data['freq'].tolist()  # 'freq' column is for x_data
    for column_name in data.columns[1:]:  # Loop through each y-data column (skip the first 'freq' column)
        y_values = data[column_name].tolist()

        # Remove None (NULL) or NaN values from both x_data and y_data together
        cleaned_data = [(x, y) for x, y in zip(x_values, y_values) 
                        if x is not None and not (isinstance(x, float) and math.isnan(x)) 
                        and y is not None and not (isinstance(y, float) and math.isnan(y))]

        # Sort the cleaned data by y_data (descending) and get the top peaks
        cleaned_data.sort(key=lambda pair: pair[1], reverse=True)  # Sort by y_value (descending)
        top_peaks = cleaned_data[:50]  # Get the top highest y-values

        # Loop through the top peaks and insert each (x, y) combination
        for x, y in top_peaks:
            # For the missing values, we insert placeholders such as NULL for mineral_name, rruff_id, etc.
            cursor.execute(f"""
            INSERT INTO {table_name} (filename, mineral_name, rruff_id, wavelength, orientation, file_number, elements, x_data, y_data)
            VALUES (?, NULL, NULL, NULL, NULL, NULL, NULL, ?, ?);
            """, (column_name, x, y))

    # Commit changes
    conn.commit()
    print(f"Top peaks from '{csv_file}' added to '{database_name}' in table '{table_name}' successfully.")

except sqlite3.OperationalError as e:
    print(f"An error occurred: {e}")

finally:
    # Close the connection
    if 'conn' in locals():
        conn.close()


Top peaks from 'MPdatabase.csv' added to 'RRUFFRaman_database.db' in table 'microplastics_data_match' successfully.
